# CHEM 1000 - Spring 2024
Prof. Geoffrey Hutchison, University of Pittsburgh

## 6 Optimizing Functions (Examples)

Chapter 6 in [*Mathematical Methods for Chemists*](http://sites.bu.edu/straub/mathematical-methods-for-molecular-science/)

By the end of this session, you should be able to:
- Understand general approaches to optimize functions either for minima or maxima (i.e. "extrema")
- Understand how to use derivatives in multiple dimensions to categorize extrema
- Using `scipy.optimize` to do numeric optimization of complicated functions
  - (We'll have more examples for both algebra/calculus and numerical optimization in recitation)

### Quick Summary

In chemistry and physics, we often want to determine the maximum or minimum value of a function of one or many variables. Examples include characterizing the minima, maxima, and saddle points on a potential energy surface.

Optimizing functions in one dimension is pretty easy, if sometimes tedious.
1. Take derivatives and find where the first derivative is zero
2. Look at the second derivatives, to categorize as a minima / maxima / inflection point
3. Then compare values of the function at those points to see if it's a local minima / max or the global minima / max.

### Many Variables

Not surprisingly, we can use a similar technique in multiple dimensions.

If we have a function $f(x,y)$ in two dimensions, then to have an extrema:

$$
\frac{\partial f}{\partial x}=0 \quad \frac{\partial f}{\partial y}=0
$$

In other words, we need to see the partial derivative with respect to **both / all** variables be zero.

We can then categorize the type of minima / maxima with the [Hessian]. (Later, we will see that this is the *determinant* of the Hessian matrix, for when we have more than 2 variables.)

$$
D=\left(\frac{\partial^{2} f}{\partial x^{2}}\right)\left(\frac{\partial^{2} f}{\partial y^{2}}\right)-\left(\frac{\partial^{2} f}{\partial x \partial y}\right)\left(\frac{\partial^{2} f}{\partial y \partial x}\right)
$$


$$
\left.\frac{\partial^{2} f}{\partial x^{2}}\right|_{\left(x^{*}, y^{*}\right)}=\left\{\begin{array}{ll}
<0 & D>0 & \text { maximum } \\
>0 & D>0 & \text { minimum } \\
 & D < 0 & \text { saddle-point }
\end{array}\right.
$$

### Example:

Let's try the so-called "butterfly potential"

$$
V(x, y)=\left((x-y)^{2}-1\right)^{2}+10 x^{2} y^{2}
$$

<img src='../images/butterfly-potential.png' />

In [ ]:
from sympy import init_session
init_session()

In [ ]:
V = ((x-y)**2-1)**2 + 10*x**2*y**2
V

In [ ]:
# okay, let's see where the partial derivative with x is zero...
diff(V, x)

In [ ]:
# now let's do y
diff(V, y)

Okay, we can't do this so easily - we have two equations and two unknowns.

Fortunately, sympy offers some nice features:
- Eq(*expression*, value) sets up an equation like $x = 0$
  - We can set our derivatives as equal to zero
- solve() can take multiple equations and multiple unknowns

In [ ]:
eq1 = Eq(diff(V, x), 0) # partial derivative with x == 0
eq2 = Eq(diff(V, y), 0) # partial derivative with y == 0
solve((eq1,eq2), (x, y)) # solve two equations for two unknowns

In [ ]:
# here's the two-variable Hessian test...
D = diff(V, x, 2)*diff(V, y, 2) - diff(V, x, y)*diff(V, y, x)
D # print it

Okay, I like when the computer does math for me...

In [ ]:
# we can use the .subs() method to substitute values (-1, 0)
D.subs([ (x, -1), (y, 0) ])

Okay, that's $D > 0$ so we need to check the second derivative (at -1, 0) to see if we're at a max or min

In [ ]:
dVdx2 = diff(V, x, x)
# use f.subs() to substitute x for -1 and y for 0
dVdx2.subs([ (x, -1), (y, 0) ])

Second derivative is positive, so this is a minimum

In [ ]:
# check (0,0)
D.subs([ (x, 0), (y, 0) ])

The origin is inconclusive - because it's something like a saddle point, but there are four wells around it.

In [ ]:
# now check (+1, 0)
D.subs([ (x, 1), (y, 0) ])

Again, this point is positive, so we need to check for a maximum or minimum. (It's another minimum)

Instead, let's look at the $\frac{\sqrt 26}{13}$ points...

In [ ]:
D.subs([ (x, sqrt(26)/13), (y, -sqrt(26)/13) ])

This point has $D < 0$ so these are saddle points

## Another Example

Let's try $$f(x,y) = x^2y - 2xy^2 + 3xy + 4$$

In [ ]:
f = x**2*y - 2*x*y**2 + 3*x*y + 4

We're going to look where the derivatives are both zero...

In [ ]:
# take derivatives
dfdx = diff(f, x)
dfdy = diff(f, y)

# solve for where x == y == 0
solve((dfdx,dfdy), (x, y))

Okay, let's get the determinant of the Hessian

In [ ]:
D = diff(f, x, 2)*diff(f, y, 2) - diff(f, x, y)*diff(f, y, x)
D

Now we check each of these points to classify them...

In [ ]:
D.subs([(x, -3), (y, 0)])

(-3, 0) is a saddle point.

In [ ]:
D.subs([(x, -1), (y, 0.5)])

Okay, let's check the second derivative to see if it's a max or min

In [ ]:
diff(f, x, 2).subs([(x, -1), (y, 0.5)])

Minimum. Next point?

In [ ]:
D.subs([(x, 0), (y, 0)])

In [ ]:
D.subs([(x, 0), (y, 1.5)])

Okay, both of these are saddle points. So the function has 3 saddle points and a minimum

### Water

We looked at a one-dimensional Lennard-Jones potential.

Here, I've calculated the O-H bond stretch in water, using the RI-MP2 method and the cc-pVTZ basis set. (Don't worry, you don't need to know what that means, but we're aiming for a relatively accurate quantum chemical method.)

<img src="../images/water-bond.png" width="500"/>

Most bonds fit well to a [Morse Potential](https://en.wikipedia.org/wiki/Morse_potential) like this. There are reasons to use other functional forms (e.g., speed), but this relatively simple form captures a lot of how real covalent bonds behave.

I also calculated the H-O-H bond angle bend using the same method. Fortunately, this (like many angle bends) fit well to a quadratic function:
          
<img src="../images/water-angle.png" width="500" />

The total potential energy will be:

$$
V = V_{bond} + V_{angle}
$$

In principle, there are "cross-terms" (e.g., stretch-bend) involving both $r$ and $\theta$ but it's always good to start with a simple model and get more complex once we understand that.

In [ ]:
# We'll declare new symbols for now
r, theta = symbols("r theta")
bond = 265.79*(1 - exp((-2.2534)*(r - 0.96242)))**2
angle = 167.16 - 3.217*theta + 0.01548*theta**2
V = bond + angle

In [ ]:
# let's solve for the derivative w.r.t. theta equal to zero
solve( diff(angle, theta) )

In [ ]:
diff(bond, r)

In [ ]:
# Don't run this, it will go forever
solve(diff(bond, r))

Oof, that's a hard one to solve. We know that the minimum value is around 0.96Å, so we can try this numerically:

In [ ]:
dr = diff(bond, r)
lengths = [0.93, 0.94, 0.95, 0.96, 0.97, 0.98]
for l in lengths:
    print(l, round(bond.subs(r, l), 4), round(dr.subs(r, l), 4))

Clearly 0.96Å is very close. Notice that the partial derivative goes from negative to positive between 0.96Å and 0.97Å.

We can interpolate:

In [ ]:
for digit in range(0, 10):
    l = 0.96 + digit/1000
    print(l, round(bond.subs(r, l), 4), round(dr.subs(r, l), 4))

I can go further, but ~0.9625Å seems like a fairly accurate estimate of the optimal bond length.

I picked this example because sometimes we want to use several methods (e.g., both calculus and numerical methods) to solve problems.

We discussed using `scipy.optimize.minimize` to solve numeric problems:

In [ ]:
import numpy as np
# it also has a bunch of minimize methods
from scipy.optimize import minimize

# we have to define the function to evaluate
def water(parameters):
    r, theta = parameters
    bond = 265.79*(1 - np.exp((-2.2534)*(r - 0.96242)))**2
    angle = 167.16 - 3.217*theta + 0.01548*theta**2
    return bond + angle

# and here's how we call it, e.g. r = 0.98A and theta = 108°
water([0.98, 108])

The `scipy.optimize` minimize() function has a bunch of related methods. The most useful for most purposes:

* `method ='CG'` for [conjugate gradients](https://en.wikipedia.org/wiki/Conjugate_gradient_method) 
   * like "skiing downhill"
   * calculates gradients / derivatives using numeric approximation if needed 
   * .. steps "downhill"
   * calculates new gradients .. and continues but not always exactly in the steepest direction
   * finishes when gradients / derivatives are "close to zero"
* `method = 'BFGS'` for [BFGS algorithm](https://en.wikipedia.org/wiki/Broyden–Fletcher–Goldfarb–Shanno_algorithm)
   * calculates an approximation to the Hessian matrix
   * also uses gradients / derivatives using numeric approximation if needed
   * usually faster / fewer steps than CG
   * a bit harder to code if you're writing it yourself
   * also finishes when gradients are close to zero .. and Hessian has no negative values

In [ ]:
# we need to start with initial parameters
x0 = [0.96, 108]
# CG = "conjugate gradients" minimize
optima = minimize(water, x0, method='CG', options={'disp': True})

In [ ]:
# here's how we get the parameters:
print(optima.x)
# and here's the value of the function at the minimum parameters:
print("energy = ", optima.fun)

In [ ]:
# use the same starting parameters
optima = minimize(water, x0, method='BFGS', options={'disp': True})

In [ ]:
print(optima.x)
print("energy = ", optima.fun)

Notice that BFGS used many fewer steps (4 vs. 19) and only 32 function evaluations (vs. 212 for CG).

In other words, if you can, BFGS is a really, really good numeric minimization method.

Why would you use conjugate gradients instead? Well, if you're writing the code, BFGS is more complicated. Writing CG is pretty easy .. just calculate the gradients and figure out how far of a step to take. Repeat.

BFGS is harder and uses more memory because it builds up an approximation to the Hessian.

So if I don't have `scipy.optimize` and I'm writing code myself.. I might use CG just because I'm lazy and I want the computer to do the work for me.

One last caveat.

<div class="alert alert-block alert-warning">
Numeric optimization methods will only find <strong>local</strong> minima (or maxima) near your initial point. So if you want to find <emph>all</emph> minima or maxima, you need to sample multiple initial points.
</div

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>